# Eksplorasi dan Pemodelan untuk Deteksi Anomali DNS

Tujuan notebook ini adalah untuk:
1. Memuat dan memahami data log DNS.
2. Melakukan feature engineering untuk mengekstrak sinyal dari data.
3. Melatih model Isolation Forest untuk mendeteksi anomali.
4. Menganalisis hasil deteksi.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import fungsi dari file utils kita
import sys
sys.path.append('../src')
from analysis_utils import feature_engineering, train_and_predict

# Atur style plot
sns.set_theme(style="whitegrid")

In [ ]:
# Muat data dari file CSV
df = pd.read_csv('../data/dns_logs.csv')
print("Data Awal:")
df.head()

## 2. Feature Engineering
Kita akan menambahkan fitur-fitur yang sudah kita definisikan (`query_length`, `query_entropy`, `subdomain_count`) menggunakan fungsi dari `analysis_utils`.

In [ ]:
# Terapkan feature engineering
df_featured = feature_engineering(df)
print("Data dengan Fitur Baru:")
df_featured.head()

## 3. Eksplorasi Data (EDA)
Mari kita visualisasikan fitur-fitur baru untuk melihat apakah anomali terlihat menonjol.

In [ ]:
# Visualisasi distribusi entropy
plt.figure(figsize=(10, 6))
sns.histplot(df_featured['query_entropy'], bins=20, kde=True)
plt.title('Distribusi Entropy dari Query DNS')
plt.xlabel('Shannon Entropy')
plt.ylabel('Frekuensi')
plt.show()

# Query dengan entropy tinggi kemungkinan adalah anomali.

In [ ]:
# Visualisasi panjang query vs entropy
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_featured, x='query_length', y='query_entropy')
plt.title('Panjang Query vs. Entropy')
plt.xlabel('Panjang Query')
plt.ylabel('Shannon Entropy')
plt.show()

# Titik-titik di kanan atas adalah kandidat kuat anomali.

## 4. Pemodelan dan Deteksi
Sekarang kita latih model `IsolationForest` dan lihat apa yang dideteksinya.

In [ ]:
# Tentukan kolom fitur yang akan digunakan
feature_cols = ['query_length', 'query_entropy', 'subdomain_count']

# Latih model dan dapatkan prediksi
predictions = train_and_predict(df_featured, feature_cols)
df_featured['is_anomaly'] = predictions

# Tampilkan hasil deteksi
anomalies = df_featured[df_featured['is_anomaly'] == -1]

print("\n--- ANOMALI TERDETEKSI ---")
print(anomalies)